In [1]:
import pandas as pd
import numpy as np
import torch

from sklearn.preprocessing import MinMaxScaler

import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

# Import nn.functional
import torch.nn.functional as F


In [2]:
df = pd.read_csv('3_dataset.csv')
print(df.shape)
df.head()

(1401, 24)


,venue,innings,batting_team,bowling_team,bat1,bat2,bat3,bat4,bat5,bat6,...,bow1,bow2,bow3,bow4,bow5,bow6,runs_off_bat,extras,Total_score,player_dismissed
0,14,1,2,5,410,73,378.0,0.0,0.0,0.0,...,318,511,18.0,0.0,0.0,0.0,51,10,61,1
1,14,2,5,2,344,493,481.0,190.0,99.0,290.0,...,21,168,19.0,0.0,0.0,0.0,19,7,26,4
2,21,1,0,3,324,278,268.0,0.0,0.0,0.0,...,67,399,202.0,0.0,0.0,0.0,50,3,53,1
3,21,2,3,0,206,202,0.0,0.0,0.0,0.0,...,187,288,0.0,0.0,0.0,0.0,61,2,63,1
4,9,1,6,1,463,505,435.0,248.0,0.0,0.0,...,151,54,269.0,0.0,0.0,0.0,38,2,40,2


In [9]:
inputs_df = df[['venue','innings','batting_team', 'bowling_team', 'bat1', 'bat2','bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10', 'bow1','bow2', 'bow3', 'bow4', 'bow5', 'bow6']]#.to_numpy() #dtype=float)

targets_df = df[['Total_score']]#.to_numpy()#dtype=float)

print(inputs_df.shape,targets_df.shape)
inputs_df.head()



(1401, 20) (1401, 1)


,venue,innings,batting_team,bowling_team,bat1,bat2,bat3,bat4,bat5,bat6,bat7,bat8,bat9,bat10,bow1,bow2,bow3,bow4,bow5,bow6
0,14,1,2,5,410,73,378.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,318,511,18.0,0.0,0.0,0.0
1,14,2,5,2,344,493,481.0,190.0,99.0,290.0,0.0,0.0,0.0,0.0,21,168,19.0,0.0,0.0,0.0
2,21,1,0,3,324,278,268.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67,399,202.0,0.0,0.0,0.0
3,21,2,3,0,206,202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,187,288,0.0,0.0,0.0,0.0
4,9,1,6,1,463,505,435.0,248.0,0.0,0.0,0.0,0.0,0.0,0.0,151,54,269.0,0.0,0.0,0.0


In [10]:
targets_df.head()

,Total_score
0,61
1,26
2,53
3,63
4,40


# Normalize data:

In [11]:

mms_input_obj = MinMaxScaler()
df_input_norm = pd.DataFrame(mms_input_obj.fit_transform(inputs_df),columns=inputs_df.columns)
df_input_norm.head()

,venue,innings,batting_team,bowling_team,bat1,bat2,bat3,bat4,bat5,bat6,bat7,bat8,bat9,bat10,bow1,bow2,bow3,bow4,bow5,bow6
0,0.43750,0.0,0.285714,0.714286,0.803150,0.140594,0.739726,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.621569,1.000000,0.035225,0.0,0.0,0.0
1,0.43750,1.0,0.714286,0.285714,0.673228,0.972277,0.941292,0.372549,0.194118,0.574257,0.0,0.0,0.0,0.0,0.039216,0.324803,0.037182,0.0,0.0,0.0
2,0.65625,0.0,0.000000,0.428571,0.633858,0.546535,0.524462,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.129412,0.779528,0.395303,0.0,0.0,0.0
3,0.65625,1.0,0.428571,0.000000,0.401575,0.396040,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.364706,0.561024,0.000000,0.0,0.0,0.0
4,0.28125,0.0,0.857143,0.142857,0.907480,0.996040,0.851272,0.486275,0.000000,0.000000,0.0,0.0,0.0,0.0,0.294118,0.100394,0.526419,0.0,0.0,0.0


In [12]:
mms_output_obj = MinMaxScaler()
df_output_norm = pd.DataFrame(mms_output_obj.fit_transform(targets_df),columns=targets_df.columns)
df_output_norm.head()

,Total_score
0,0.572816
1,0.233010
2,0.495146
3,0.592233
4,0.368932


In [18]:
inputs_np = df_input_norm.to_numpy()
targets_np = df_output_norm.to_numpy()

In [19]:
inputs = torch.from_numpy(inputs_np).float()
targets = torch.from_numpy(targets_np).float()
targets = targets.view(1401,1)
print(inputs)
print(targets)

tensor([[0.4375, 0.0000, 0.2857,  ..., 0.0000, 0.0000, 0.0000],
        [0.4375, 1.0000, 0.7143,  ..., 0.0000, 0.0000, 0.0000],
        [0.6562, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.8438, 1.0000, 1.0000,  ..., 0.5558, 0.0000, 0.0000],
        [0.2188, 0.0000, 0.1429,  ..., 0.6047, 0.0000, 0.0000],
        [0.2188, 1.0000, 0.5714,  ..., 0.5558, 0.0909, 0.0000]])
tensor([[0.5728],
        [0.2330],
        [0.4951],
        ...,
        [0.4563],
        [0.3786],
        [0.5728]])


In [20]:
print(inputs.shape)
print(targets.shape)

torch.Size([1401, 20])
torch.Size([1401, 1])


In [21]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
len(train_ds)

1401

In [22]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)


In [23]:
for i,j in train_dl:
    print(i.shape,j.shape)
    print(i)
    print(j)
    break

torch.Size([5, 20]) torch.Size([5, 1])
tensor([[0.7500, 1.0000, 0.0000, 0.8571, 0.5236, 0.5050, 0.8160, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.8510, 0.6575, 0.8689, 0.0000,
         0.0000, 0.0000],
        [0.4375, 0.0000, 0.5714, 0.7143, 0.6752, 0.8554, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2725, 0.6890, 0.6595, 0.3718,
         0.0000, 0.0000],
        [1.0000, 0.0000, 0.2857, 0.5714, 0.2854, 0.3723, 0.8415, 0.5431, 0.9902,
         0.7267, 0.0000, 0.0000, 0.0000, 0.0000, 0.2647, 0.5453, 0.3209, 0.8219,
         0.0000, 0.0000],
        [0.6562, 1.0000, 0.4286, 0.0000, 0.6378, 0.0455, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6647, 0.9154, 0.3562, 0.0000,
         0.0000, 0.0000],
        [0.2812, 0.0000, 0.1429, 0.2857, 0.2146, 0.9525, 0.2564, 0.4804, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1294, 0.6417, 0.3718, 0.8454,
         0.0000, 0.0000]])
tensor([[0.3301],
  

In [24]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[0.2812, 1.0000, 0.2857, 0.1429, 0.7421, 0.2871, 0.5401, 0.7941, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2373, 0.6024, 0.0568, 0.0000,
         0.0000, 0.0000],
        [0.2812, 1.0000, 0.8571, 1.0000, 0.6732, 0.0752, 0.8513, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2725, 0.3248, 0.6027, 0.0000,
         0.0000, 0.0000],
        [0.6875, 0.0000, 0.5714, 1.0000, 0.2618, 0.8554, 0.4188, 0.7078, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4451, 0.2697, 0.3288, 0.6027,
         0.0000, 0.0000],
        [0.3750, 0.0000, 0.0000, 0.2857, 0.2618, 0.1406, 0.8160, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.8804, 0.8445, 0.6497, 0.0000,
         0.0000, 0.0000],
        [1.0000, 0.0000, 0.5714, 0.1429, 0.6339, 0.3604, 0.5949, 0.7078, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4059, 0.1772, 1.0000, 0.0000,
         0.0000, 0.0000]])
tensor([[0.4466],
        [0.4078],
        [0.3786],
     

In [25]:
#torch.autograd.detect_anomaly(True)

# Define model
model = nn.Linear(20,1)
print(model.weight.shape)
print(model.bias.shape)

torch.Size([1, 20])
torch.Size([1])


In [26]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.2012, -0.1648,  0.0150,  0.0070, -0.0349, -0.2039,  0.0166,  0.0358,
           0.0949, -0.0835,  0.0623, -0.0488,  0.1865, -0.1277, -0.0991, -0.1090,
           0.0785,  0.1058,  0.0039,  0.2139]], requires_grad=True),
 Parameter containing:
 tensor([0.1230], requires_grad=True)]

In [27]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-0.1681],
        [-0.3760],
        [-0.1977],
        ...,
        [-0.2499],
        [ 0.1233],
        [-0.2730]], grad_fn=<AddmmBackward>)

# Loss Function


In [28]:
# Define loss function
loss_fn = nn.MSELoss()

loss = loss_fn(model(inputs), targets)
print(loss)

tensor(0.4204, grad_fn=<MseLossBackward>)


# Optimizer

In [29]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-1)

# Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function fit that trains the model for a given number of epochs.

In [30]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        # Train with batches of data
        for xb,yb in train_dl:
            # 1. Generate predictions
            pred = model(xb)
            #print(pred)
            #import pdb;pdb.set_trace()
            #print(pred);import pdb;pdb.set_trace()
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            # 3. Compute gradients
            loss.backward()
            # 4. Update parameters using gradients
            opt.step()
            # 5. Reset the gradients to zero
            opt.zero_grad()
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {}'.format(epoch+1, num_epochs, loss.item()))

In [31]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 0.0032354933209717274
Epoch [20/100], Loss: 0.0006307504372671247
Epoch [30/100], Loss: 0.001007897430099547
Epoch [40/100], Loss: 0.010409107431769371
Epoch [50/100], Loss: 0.005613001529127359
Epoch [60/100], Loss: 0.13057604432106018
Epoch [70/100], Loss: 0.001513162744231522
Epoch [80/100], Loss: 0.01755564846098423
Epoch [90/100], Loss: 0.013629505410790443
Epoch [100/100], Loss: 0.021487638354301453


In [32]:
# Generate predictions
preds = model(inputs)
preds

tensor([[0.3759],
        [0.2808],
        [0.4122],
        ...,
        [0.3344],
        [0.2807],
        [0.4078]], grad_fn=<AddmmBackward>)

In [50]:
#pd.DataFrame(,index=False,columns='Actual Score in 6-overs')
mms_output_obj.inverse_transform(targets)

array([[61.00000036],
       [26.00000022],
       [52.99999875],
       ...,
       [49.00000101],
       [40.9999994 ],
       [61.00000036]])

In [52]:
pred_df = pd.DataFrame(mms_output_obj.inverse_transform(preds.detach().numpy()),columns=['Predicted Score in 6-Overs'])

In [56]:
pd.concat([inputs_df,targets_df,pred_df],axis=1).to_csv('model_pred.csv')